In [1]:
import pandas as pd
import numpy as np
import re

In [220]:
df_map = pd.read_csv('24_regions.csv', index_col = 0)

In [221]:
df = pd.read_excel('minosvita_df.xlsx')

In [242]:
"""
РЕГУЛЯРНІ ВИРАЗИ ТА ФІЛЬТРИ
"""

reg_number = '(№\s*\d+)' #витягує число перед яким є символ №
reg_quotes = """["'«](.*?)["'»]{1}?""" #витягує текст в лапках трьох видів
lot_regex = '(((\DЗОШ\D)|(\Dзош\D)|(\DСШ\D)|(\DДНЗ\D)|(\DДНЗ\D)).(\d+))' #витягує номер поруч з ключовими словами
reg_city = "(м\.\w+)" # витягує назву міста після "м."

#набір ключових слів-фільтрів для відділення шкільних закупівель
filter_keywords = ['зошщ', 'навчально-виховн' ,'школ', 'шкіль', 'гімназ', 'ліцей', 'ліцея', 'зош', 'знз', 'сш', 'осш', 'колегіум', 'нвк', 'увк', 'снвк', 'загальноосвітн', 'сзш', 'виховн', 'днз', 'садок', 'дошкільн', 'садка']

#набір ключових слів-фільтрів для кожної категорії окремо
filter_shkola = ['школ', 'шкіль',  'гімназ',  'ліцей', 'ліцея', 'зош', 'знз', 'сш', 'колегіум', 'нвк', 'увк', 'снвк', 'загальноосвітн', 'сзш', 'виховн', 'осш']
filter_shkola_name = 'Школа'

filter_sadik = ['днз', 'ДНЗ', 'садок', 'дошкільн', 'садка', 'ясла', 'ясел']
filter_sadik_name = 'ДНЗ'

filter_sport = ['ДЮСШ', 'юнацька спортив', 'дюсш', 'СДЮСШОР', 'дитячо-спортивна']
filter_sport_name = 'ДЮСШ'

filter_music = ['музич', "дмш"]
filter_music_name = 'Музична школа'

filter_art = ['мистецтв', 'дхш']
filter_art_name = 'Школа мистецтв'

filter_hud = ['худож']
filter_hud_name = 'Художня школа'

filter_poza = ['позашкільн', 'міжшкільн', 'МСВЗШ', 'спеціальна вечірня']
filter_poza_name = 'Позашкільне'

filter_inshe = 'Інше'

In [235]:
# Створення таблиці з головним органом і відповідним містом

regions = ['Суми', 'Київ', 'Полтава', 'Харків', 'Чернігів', 'Вінниця', 'Львів', 'Миколаїв', 'Дніпро',
           'Чернівці', 'Запоріжжя', 'Одеса', 'Черкаси', 'Херсон', 'Рівне', 'Тернопіль', 'Ужгород', 
           'Луцьк', 'Івано-Франківськ', 'Житомир', 'Кропивницький', 'Хмельницький']

gol_organ = ['Сумська область',
 'м.Київ',
 'Полтавська область',
 'Харківська область',
 'Чернігівська область',
 'Вінницька область',
 'Львівська область',
 'Миколаївська область',
 'Дніпровська область',
 'Чернівецька область',
 'Запорізька область',
 'Одеська область',
 'Черкаська область',
 'Херсонська область',
 'Рівненська область',
 'Тернопільська область',
 'Ужгородська область',
 'Луцька область',
 'Івано-Франківська область',
 'Житомирська область',
 'Кропивницька область',
 'Хмельницька область']

city_organ = pd.DataFrame(list(gol_organ), columns=['gol_organ'])
city_organ['city'] = pd.Series(regions)
city_organ = city_organ[['gol_organ', 'city']]

# city_organ.sample()

In [330]:
"""
ФУНКЦІЯ, ЯКА ВИТЯГУЄ ІМЕНА ШКІЛ І ПРИВОДИТЬ ДО ВІДПОВІДНОСТІ ФОРМАТУ ЗАПИСУ В ТАБЛИЦІ МАП

"""

def fix_table():    
    
    df_fixed = df.copy()
    df_fixed['type'] = pd.np.where(df_fixed["name"].str.contains('|'.join(filter_sport), case = False), filter_sport_name,
                            pd.np.where(df_fixed["name"].str.contains('|'.join(filter_music), case = False), filter_music_name,
                            pd.np.where(df_fixed["name"].str.contains('|'.join(filter_art), case = False), filter_art_name,
                            pd.np.where(df_fixed["name"].str.contains('|'.join(filter_hud), case = False), filter_hud_name,
                            pd.np.where(df_fixed["name"].str.contains('|'.join(filter_poza), case = False), filter_poza_name,
                            pd.np.where(df_fixed["name"].str.contains('|'.join(filter_sadik), case = False), filter_sadik_name,
                            pd.np.where(df_fixed["name"].str.contains('|'.join(filter_shkola), case = False), filter_shkola_name,
                                         filter_inshe)))))))
    
    df_fixed = df_fixed[(df_fixed['type'] != "Інше") & (df_fixed['type'] != "Позашкільне")]
    
    df_fix_temp_num = df_fixed.copy()
    df_fix_temp_num['name_fixed_num'] = df_fix_temp_num['name'].str.extract(reg_number, expand = False)
    

    df_fix_temp_str = df_fix_temp_num[df_fix_temp_num['name_fixed_num'].isnull()]
    df_fix_temp_num = df_fix_temp_num[df_fix_temp_num['name_fixed_num'].notnull()]
    df_fix_temp_num['name_fixed_num'] = df_fix_temp_num['name_fixed_num'].str.replace("№ ", "№")

    df_fix_temp_str['name_fixed_num'] = df_fix_temp_str['name'].str.extract(reg_quotes, expand = False)
    df_fix_temp_str = df_fix_temp_str[df_fix_temp_str['name_fixed_num'].notnull()]
    
    frames = [df_fix_temp_num, df_fix_temp_str]
    df_reborn = pd.concat(frames)
    
    
    """
    ВИТЯГУВАННЯ МІСТ З АДРЕС
    """
    df_reborn['full'] = df_reborn['full'].str.replace('м. ','м.')
    df_reborn['city_new'] = df_reborn['full'].str.extract(reg_city, expand = False)
    df_reborn['city_new'] = df_reborn['city_new'].str.replace('м\.','')
    df_reborn['city_new'] = df_reborn['city_new'].str.replace('Івано','Івано-Франківськ')
    
    print("Було закладів: ", len(df_reborn))
    df_reborn = df_reborn[df_reborn.city_new.str.contains('|'.join(regions), na = False, case=False)]   
    
    
    """
    СТВОРЕННЯ КОЛОНКИ З МІСТОМ І ШКОЛОЮ
    """ 
    df_reborn['city_fixed'] = df_reborn['city_new'] + " " + df_reborn['type'] + " " + df_reborn['name_fixed_num']
    
    df_reborn = df_reborn.replace(np.nan, '', regex=True)
    
    df_reborn['street'] = df_reborn['street'].astype(str)
    df_reborn['street2'] = df_reborn['street2'].astype(str)

    
    """
    СТВОРЕННЯ КОЛОНКИ З АЛЬТЕРНАТИВНОЮ АДРЕСОЮ З МІСТОМ І ВУЛИЦЕЮ
    """
    df_reborn['address_fixed'] = df_reborn['city_new'] + " " + df_reborn['street'] + " " + df_reborn['street2']
    
    df_reborn['address_fixed'] = df_reborn['address_fixed'].str.replace("  "," ")
    
    
    """
    ПІДРАХУНОК КІЛЬКОСТІ СЛІВ У КОЛОНЦІ З НАЗВОЮ ШКІЛ. 
    ЯКЩО БІЛЬШЕ 1, ТО ШВИДШЕ ЗА ВСЕ, ЦЕ НЕ НАЗВА ЗАКЛАДУ
    """
    df_reborn['word_count'] = df_reborn['name_fixed_num'].str.split().str.len()
    df_reborn = df_reborn[(df_reborn.word_count == 1) | (df_reborn.name_fixed_num.str.contains('ім\.'))]
    
    
    """
    ВИДАЛЕННЯ ЗАЙВИХ МІСТ
    """
    bad_cities = ['Дніпрорудне','Верхньодніпровськ', 'Миколаївка', 'Новомиколаївка', 'Верхньодніпровськ', 'Чернігівка', 'Дніпровське']
    df_reborn = df_reborn[~df_reborn['city_fixed'].str.contains('|'.join(bad_cities), case = False)]
    print("Залишилось після фільтрування:", len(df_reborn)) 
    print("Міста: ", df_reborn.city_new.unique())
    
    df_reborn = df_reborn[['city_fixed', 'address_fixed', 'full_2', 'full_3']]
    
    return df_reborn

In [331]:
df_reborn = fix_table()
df_reborn.sample(10)

Було закладів:  19329
Залишилось після фільтрування: 2251
Міста:  ['Дніпро' 'Івано-Франківськ' 'Запоріжжя' 'Черкаси' 'Харків' 'Чернівці'
 'Херсон' 'Миколаїв' 'Вінниця' 'Хмельницький' 'Львів' 'Кропивницький'
 'Суми' 'Одеса' 'Житомир' 'Рівне' 'Луцьк' 'Тернопіль' 'Ужгород' 'Полтава'
 'Чернігів' 'Київ']


In [335]:
df_reborn.to_csv('minosvity_fixed.csv', index = True, header=True)